# Western Cape Dam Levels
Data has been downloaded from City of Cape Town Open Data Portal:  

## GIS
City of Cape Town Corporate GIS  
https://odp-cctegis.opendata.arcgis.com/datasets/cctegis::bulk-water-dams-1/explore?location=-33.865920%2C19.071866%2C11.86  
<br>
Bulk Water dams (Bulk_Water_Dams.geojson):  
https://odp-cctegis.opendata.arcgis.com/datasets/cctegis::bulk-water-dams-1/explore?location=-33.865920%2C19.071866%2C11.86  
<br>
## Timeseries
Dam Levels (Dam_Levels_from_2012.csv):  
https://odp-cctegis.opendata.arcgis.com/datasets/cctegis::dam-levels-from-2012/about  
<br>
(Not Useful)  
Water Consumption (data/Water_consumption.xlsx):  
https://odp-cctegis.opendata.arcgis.com/documents/cctegis::water-consumption-1/about  
<br>
(Not Useful)  
Inland Water Quality Monthly Summary Report (Inland_WQ_Summary_Report.pdf):  
https://odp-cctegis.opendata.arcgis.com/documents/cctegis::inland-water-quality-monthly-summary-report/about  
<br>
(Not Useful)  
Rainfall Data From 2000 (Rainfall_Data_2000_to_2024.csv):  
https://odp-cctegis.opendata.arcgis.com/datasets/cctegis::rainfall-data-from-2000-1/explore  

## Weather Data
meteostat python package 

## Poulations data
https://www.macrotrends.net/global-metrics/cities/22481/cape-town/population

### STEP 1: Dam Levels Data

In [1]:
import pandas as pd
import geopandas as gpd
import json
import numpy as np
from datetime import datetime
import os
from shapely.geometry import Point

# Load dam polygon GeoJSON
gdf = gpd.read_file("data/2025-05-01/Bulk_Water_Dams.geojson")

# Load dam levels CSV, set date format
df = pd.read_csv("data/2025-06-06/Dam_Levels_from_2012.csv", encoding="ISO-8859-1")
df['DATE'] = df['DATE'].str.replace('Sept', 'Sep', regex=False)
df['DATE'] = pd.to_datetime(df['DATE'], format='%d-%b-%y')

# Clean column names
df.columns = df.columns.str.strip().str.replace(r"\s+", "", regex=True).str.lower()

# Mapping from NAME in GeoJSON to CSV prefix (lowercase, no spaces)
dam_name_mapping = {
    "Woodhead": "woodhead",
    "Hely-Hutchinson": "hely-hutchinson",
    "Lewis Gay": "lewisgay",
    "Kleinplaats": "kleinplaats",
    "Victoria": "victoria",
    "Alexandra": "alexandra",
    "De Villiers": "devilliers",
    "Steenbras Lower": "steenbraslower",
    "Steenbras Upper": "steenbrasupper",
    "Voëlvlei": "voëlvlei",
    "Wemmershoek": "wemmershoek",
    "Theewaterskloof": "theewaterskloof",
    "Berg River": "bergriver",
    "Land-en-Zeezicht Dam": "land-enzeezicht",
    "Big 5 Total": "totalstored-big5",
    "Big 6 Total": "totalstored-big6"
}

def build_timeseries(prefix):
    # Find all columns related to this dam (that start with the prefix)
    prefix_cols = [col for col in df.columns if col.startswith(prefix)]

    def find_col(keyword):
        # Look for a column that contains the keyword (case-insensitive)
        matches = [col for col in prefix_cols if keyword in col]
        return matches[0] if matches else None

    # Find matching columns
    height_col = find_col("height")
    storage_col = find_col("storage")
    current_col = find_col("current")
    last_year_col = find_col("lastyear")

    # If we find no relevant columns, return empty
    if not any([height_col, storage_col, current_col, last_year_col]):
        return [], None

    # Build DataFrame
    cols = {'date': 'date'}
    if height_col: cols[height_col] = 'height_m'
    if storage_col: cols[storage_col] = 'storage_ml'
    if current_col: cols[current_col] = 'percent_full'
    if last_year_col: cols[last_year_col] = 'last_year_percent_full'

    col_keys = list(cols.keys())
    if 'date' not in col_keys:
        col_keys = ['date'] + col_keys
    ts = df[col_keys].copy()
    # ts['date'] = pd.to_datetime(ts['date']).dt.strftime('%Y-%m-%d')  # ensure datetime
    ts.rename(columns=cols, inplace=True)

    # Ensure numeric columns are truly numeric
    for col in ['height_m', 'storage_ml', 'percent_full', 'last_year_percent_full']:
        if col in ts.columns:
            ts[col] = pd.to_numeric(ts[col], errors='coerce')

    # format nulls
    ts = ts.where(pd.notnull(ts), None)

    return ts


# Create output containers
dam_ts_daily = {}
dam_ts_monthly = {}
dam_ts_yearly = {}

for dam_name, prefix in dam_name_mapping.items():
    df_ts = build_timeseries(prefix)
    if df_ts is None or df_ts.empty:
        print('no data set found')
        continue

    df_ts['date'] = pd.to_datetime(df_ts['date'])

    # DAILY
    df_ts_sorted = df_ts.sort_values('date')
    df_ts_sorted = df_ts_sorted.round(2)
    df_ts_sorted['date'] = df_ts_sorted['date'].dt.strftime('%Y-%m-%d')
    dam_ts_daily[prefix] = df_ts_sorted.where(pd.notnull(df_ts_sorted), None).to_dict(orient='records')

    # MONTHLY
    monthly = df_ts.resample('ME', on='date').mean(numeric_only=True).reset_index()
    monthly = monthly.round(2)
    monthly['date'] = monthly['date'].dt.strftime('%Y-%m')
    dam_ts_monthly[prefix] = monthly.where(pd.notnull(monthly), None).to_dict(orient='records')

    # YEARLY
    yearly = df_ts.resample('YE', on='date').mean(numeric_only=True).reset_index()
    yearly = yearly.round(2)
    yearly['date'] = yearly['date'].dt.strftime('%Y')
    dam_ts_yearly[prefix] = yearly.where(pd.notnull(yearly), None).to_dict(orient='records')


# Update GeoJSON properties with centroid
for i, row in gdf.iterrows():
    if row["geometry"]:
        centroid = row["geometry"].centroid
        gdf.at[i, 'centroid'] = Point(centroid.x, centroid.y)
    else:
        gdf.at[i, 'centroid'] = None

# Clean NaNs recursively
def clean_nans(obj):
    if isinstance(obj, dict):
        return {k: clean_nans(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_nans(v) for v in obj]
    elif isinstance(obj, float) and (np.isnan(v := obj)):
        return None
    return obj

# Save Timeseries data
os.makedirs("output/timeseries", exist_ok=True)

with open("output/timeseries/dam_levels_daily.json", "w") as f:
    json.dump(clean_nans(dam_ts_daily), f, indent=2)

with open("output/timeseries/dam_levels_monthly.json", "w") as f:
    json.dump(clean_nans(dam_ts_monthly), f, indent=2)

with open("output/timeseries/dam_levels_yearly.json", "w") as f:
    json.dump(clean_nans(dam_ts_yearly), f, indent=2)

# Save enriched GeoJSON
os.makedirs("output", exist_ok=True)
gdf.to_file("output/Bulk_Water_Dams_Enriched.geojson", driver='GeoJSON')


### STEP 2: Weather Data

In [2]:
# import os
# import json
# import numpy as np
# from datetime import datetime
# import pandas as pd
from meteostat import Point, Daily

def clean_nans(obj):
    if isinstance(obj, dict):
        return {k: clean_nans(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_nans(v) for v in obj]
    elif isinstance(obj, float) and (np.isnan(v := obj)):
        return None
    return obj


# Define Cape Town coordinates
cape_town = Point(-33.9258, 18.4232)

# Time range: last 20 years
start = datetime(2000, 1, 1)
end = datetime.today()

# Fetch daily weather data
data = Daily(cape_town, start, end)
data = data.fetch()

# Filter for average temperature and precipitation
df = data[['tavg', 'prcp']].copy()

# resample monthly
monthly = df.resample('ME').mean()
monthly['prcp'] = df['prcp'].resample('ME').sum()

# resample yearly
yearly = df.resample('YE').mean()
yearly['prcp'] = df['prcp'].resample('YE').sum()

# Add date columns for export
df = df.copy()
df = df.sort_index()
df['date'] = df.index.strftime('%Y-%m-%d')
daily_out = df.reset_index(drop=True)[['date', 'tavg', 'prcp']]

monthly = monthly.copy()
monthly['date'] = monthly.index.strftime('%Y-%m')
monthly_out = monthly.reset_index(drop=True)[['date', 'tavg', 'prcp']]

yearly = yearly.copy()
yearly['date'] = yearly.index.strftime('%Y')
yearly_out = yearly.reset_index(drop=True)[['date', 'tavg', 'prcp']]

# Ensure output directory exists
os.makedirs("output/timeseries", exist_ok=True)

with open("output/timeseries/cape_town_rainfall_daily.json", "w") as f:
    json.dump(clean_nans(daily_out.to_dict(orient='records')), f, indent=2)

with open("output/timeseries/cape_town_rainfall_monthly.json", "w") as f:
    json.dump(clean_nans(monthly_out.to_dict(orient='records')), f, indent=2)

with open("output/timeseries/cape_town_rainfall_yearly.json", "w") as f:
    json.dump(clean_nans(yearly_out.to_dict(orient='records')), f, indent=2)


### STEP 3: Population Data

In [29]:
import json, os

annualPpopulationData = 'data/2025-05-19/Cape-Town-Population-Total-Population-By-Year-2025-05-17-22-32.csv'

# Load dam levels CSV
df = pd.read_csv(annualPpopulationData)
df.rename(columns={'Unnamed: 0': 'year'}, inplace=True)
# df['year'] = pd.to_datetime(df['year'])
df.columns = ['year', 'population']

df = df[['year', 'population']].dropna().sort_values('year')

os.makedirs("output/timeseries", exist_ok=True)

with open("output/timeseries/cape_town_population_yearly.json", "w") as f:
    json.dump(df.to_dict(orient='records'), f, indent=2)
